<a href="https://colab.research.google.com/github/jehanp/android-studio-tutorials/blob/master/YOLOv3_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Our YOLOv3 implementation calls for this Keras version
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
#tensorFlow
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
# Verify our version is correct
!python -c 'import keras; print(keras.__version__)'

2.9.0


In [1]:
# Next, we'll grab all the code from our repository of interest 
!git clone https://github.com/roboflow-ai/keras-yolo3

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 169, done.
remote: Total 169 (delta 0), reused 0 (delta 0), pack-reused 169
Receiving objects: 100% (169/169), 172.74 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
# here's what we cloned (also, see "Files" in the left-hand Colab pane)
%ls

keras-yolo3/  sample_data/


In [4]:
# change directory to the repo we cloned
%cd keras-yolo3/

/content/keras-yolo3


In [5]:
# show the contents of our repo
%ls

coco_annotation.py  LICENSE              Tutorial.ipynb     yolov3-tiny.cfg
convert.py          model_data/          voc_annotation.py  yolo_video.py
darknet53.cfg       README.md            yolo3/
font/               train_bottleneck.py  yolo.py
kmeans.py           train.py             yolov3.cfg


In [6]:
#Import Dataset from RoboFlow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="810yZ9ZM1qbk4Y0oaMzh")
project = rf.workspace("yolocustom-6n8ap").project("vehicles-tyidn")
dataset = project.version(2).download("yolokeras")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Vehicles-2 in yolokeras:: 100%|██████████| 1048/1048 [00:00<00:00, 1520.85it/s]


In [7]:
%ls

coco_annotation.py  LICENSE              Tutorial.ipynb     yolov3.cfg
convert.py          model_data/          Vehicles-2/        yolov3-tiny.cfg
darknet53.cfg       README.md            voc_annotation.py  yolo_video.py
font/               train_bottleneck.py  yolo3/
kmeans.py           train.py             yolo.py


In [8]:
# change directory into our export folder from Roboflow
%cd Vehicles-2/

/content/keras-yolo3/Vehicles-2


In [9]:
# show what came with the Roboflow export
%ls

README.dataset.txt  README.roboflow.txt  test/  train/  valid/


In [10]:
# move everything from the Roboflow export to the root of our keras-yolo3 folder
%mv * ../

In [11]:
# change directory back to our 
%cd ..

/content/keras-yolo3


In [15]:
# show that all our images, _annotations.txt, and _classes.txt made it to our root directory
%ls

coco_annotation.py  model_data/          train_bottleneck.py  yolo.py
convert.py          README.dataset.txt   train.py             yolov3.cfg
convert_v2.py       README.md            Tutorial.ipynb       yolov3-tiny.cfg
darknet53.cfg       README.roboflow.txt  valid/               yolov3.weights
font/               report.txt           Vehicles-2/          yolo_video.py
kmeans.py           test/                voc_annotation.py
LICENSE             train/               yolo3/


Set up and train our model

Next, we'll download pre-trained weighs weights from DarkNet, set up our YOLOv3 architecture with those pre-trained weights, and initiate training

In [13]:
# download our DarkNet weights 
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-12-25 05:29:37--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.6MB/s    in 11s     

2022-12-25 05:29:49 (21.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [14]:
#Upgrade TensorFLow v2
!tf_upgrade_v2 \
  --infile convert.py \
  --outfile convert_v2.py

WARNING line 245:8: *.save requires manual check. (This warning is only applicable if the code saves a tf.Keras model) Keras model.save now saves to the Tensorflow SavedModel format by default, instead of HDF5. To continue saving to HDF5, add the argument save_format='h5' to the save() function.
TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 1 files
Detected 1 issues that require attention
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
File: convert.py
--------------------------------------------------------------------------------
convert.py:245:8: WARNING: *.save requires manual check. (This warning is only applicable if the code saves a tf.Keras model) Keras model.save now saves to the Tensorflow SavedModel format by default, instead of HDF5. To continue saving to HDF5, add the argument save_format='h5' to the save() function.


Make sure to read 

In [23]:
# call a Python script to set up our architecture with downloaded pre-trained weights
!python convert_v2.py yolov3.cfg yolov3.weights model_data/yolo.h5

Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
2022-12-25 05:35:45.767276: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsing section convolutional_5
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_6
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_1
Parsing section convolutional_7
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_8
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_2
Parsing section convolutional_9
conv2d bn leaky (3, 3, 128, 256)
Pa

In [24]:
# can call this cell instead of the above
!python train.py

Traceback (most recent call last):
  File "train.py", line 12, in <module>
    from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
  File "/content/keras-yolo3/yolo3/model.py", line 9, in <module>
    from keras.layers.advanced_activations import LeakyReLU
ModuleNotFoundError: No module named 'keras.layers.advanced_activations'
